In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  

</style>

In [2]:
import datetime
import pyaurorax
import cartopy.crs
import numpy as np
from concurrent.futures import ProcessPoolExecutor

aurorax = pyaurorax.PyAuroraX()
at = aurorax.tools

### **Step 1: Download and Read Necessary Data**

In [3]:
# Download 5 minutes of RGB data from several sites
dataset_name = "TREX_RGB_RAW_NOMINAL"
start_dt = datetime.datetime(2023, 2, 24, 6, 15)
end_dt = datetime.datetime(2023, 2, 24, 6, 19)
site_uid_list = ['yknf', 'gill', 'rabb', 'luck', 'atha']
data_download_objs = {}
for site_uid in site_uid_list:
    download_obj = aurorax.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid)
    data_download_objs[site_uid] = download_obj

# Read in the data site-by-site, as we need this separation for mosaicing
data_list = []
for site_uid, download_obj in data_download_objs.items():
    data_list.append(aurorax.data.ucalgary.read(download_obj.dataset, download_obj.filenames))

### **Step 2: Download Skymaps**

In [4]:
# Get list of all skymaps in previous two years up to date of interest for each site
skymap_download_objs = {}
for site_uid in site_uid_list:
    download_obj = aurorax.data.ucalgary.download(
        "TREX_RGB_SKYMAP_IDLSAV",
        start_dt.replace(year=start_dt.year - 2),
        start_dt,
        site_uid=site_uid,
    )
    skymap_download_objs[site_uid] = download_obj

# Read and store the last list index skymap for each site, as that is most recent to the date of interest
skymaps = []
for site_uid, download_obj in skymap_download_objs.items():

    skymap = aurorax.data.ucalgary.read(download_obj.dataset, download_obj.filenames[-1])
    skymaps.append(skymap.data[0])

skymaps

[Skymap(project_uid=rgb, site_uid=yknf, imager_uid=rgb-08, site_map_latitude=62.519848, site_map_longitude=245.686966, ...),
 Skymap(project_uid=rgb, site_uid=gill, imager_uid=rgb-04, site_map_latitude=56.376724, site_map_longitude=265.356323, ...),
 Skymap(project_uid=rgb, site_uid=rabb, imager_uid=rgb-06, site_map_latitude=58.227810, site_map_longitude=256.319366, ...),
 Skymap(project_uid=rgb, site_uid=luck, imager_uid=rgb-03, site_map_latitude=51.153988, site_map_longitude=252.735260, ...),
 Skymap(project_uid=rgb, site_uid=atha, imager_uid=rgb-07, site_map_latitude=54.602631, site_map_longitude=246.355728, ...)]

### **Step 3: Prep Skymap and Image Data**

In [5]:
# prepare the skymap data
prepped_skymap = at.mosaic.prep_skymaps(skymaps, 110, n_parallel=5)
prepped_skymap

Preparing skymaps:   0%|          | 0/5 [00:00<?, ?skymap/s]

MosaicSkymap(polyfill_lat=array(dims=(5, 265440), dtype=float64), polyfill_lon=array(dims=(5, 265440), dtype=float64), elevation=array(dims=(265440,), dtype=float32), site_uid_list=['yknf', 'gill', 'rabb', 'luck', 'atha'])

In [6]:
# prepare the image data
prepped_images = at.mosaic.prep_images(data_list)
prepped_images

### **Step 4: Iteratively Generate and Save Mosaic Frames**
Here, we will utilize multiprocessing for increased efficiency.

In [ ]:
# First set up some settings for the mosaic to be applied to all frames

# define the intensity scales for each site
scale = {
    "yknf": [10, 105],
    "gill": [10, 105],
    "rabb": [10, 105],
    "luck": [10, 105],
    "atha": [10, 105],
}

# create projection
center_lat = -100.0
center_lon = 55.0
projection_obj = cartopy.crs.NearsidePerspective(central_longitude=center_lat, central_latitude=center_lon)

In [ ]:
# Let's see how many frames we will be making
print(f"Image Data consists of {len(prepped_images.timestamps)} frames.")

Image Data consists of 100 frames.


In [ ]:
# Define all image indexes (0-99)
#frame_idx_arr = np.arange(len(prepped_images.timestamps))
frame_idx_arr = np.arange(0, 0)

# We will process with 5 worker pools
n_parallel = 5
at.mosaic.create(
    prepped_images,
    prepped_skymap,
    0,
    projection_obj,
    image_intensity_scales=scale,
)

# create_args = []
# for frame_idx in frame_idx_arr:
#         create_args.append({
#                 "prepped_images": prepped_images,
#                 "prepped_skymap": prepped_skymap,
#                 "frame_idx": frame_idx,
#                 "projection"
#         })

create_args = [[prepped_images, prepped_skymap, frame_idx, projection_obj, scale] for frame_idx in frame_idx_arr]
from functools import partial


def process_mosaic_frame(args):
    m = at.mosaic.create(args[0], args[1], args[2], args[3], image_intensity_scales=args[4])
    print(m)
    return m


# mosaic_objs = []
# with ProcessPoolExecutor(max_workers=1) as executor:
#     for result in executor.map(process_mosaic_frame, create_args):
#         mosaic_objs.append(result)
#         print(result)

from concurrent.futures import as_completed

with ProcessPoolExecutor(max_workers=1) as executor:
        results = [executor.submit(process_mosaic_frame, create_arg) for create_arg in create_args]

        for f in as_completed(results):
                print(f.result())
